In [168]:
from pathlib import Path
from skimage import io
from skimage.io import find_available_plugins
from matplotlib import pyplot as plt
import PyQt5
from skimage.filters import threshold_otsu, threshold_local, median
from skimage.color import rgb2gray, gray2rgb
from skimage.util import img_as_ubyte, img_as_bool
from skimage.filters import gaussian, laplace
from skimage.morphology import erosion, dilation, opening, closing
from skimage.exposure import rescale_intensity, match_histograms
from skimage import feature
from collections import Counter
import numpy as np
from skimage.morphology import square
from skimage.measure import approximate_polygon, subdivide_polygon, find_contours
from skimage import draw
from skimage.feature import canny
from skimage.transform import probabilistic_hough_line
from skimage import filters
from skimage import measure
from skimage import transform
import math
from skimage import util 
%matplotlib qt 



In [15]:
from skimage.filters import threshold_multiotsu

def local_thresholding(img):   
#     img = rgb2gray(img)
#     img = img_as_ubyte(img)
#     height, width = img.shape
#     temp = int((height+width)/10)
#     block_size = temp if temp%2==1 else temp+1
#     offset = np.mean(img.flatten())/2
    local_thresh = threshold_local(img, 7, offset=0, method='gaussian')

    binary_local = img > local_thresh
    
    return binary_local
    
def global_thresholding(img):   
#     img = rgb2gray(img)
#     img = img_as_ubyte(img)

    global_thresh = threshold_otsu(img)
    binary_global = img > global_thresh

    return binary_global   

def global_thresholding_v2(img, rows=3, columns=2):   
    height, width = img.shape
    
    height_part = int(height/rows)
    width_part = int(width/columns)
    
    for row in range(rows):
        for col in range(columns):
#             sector = img[row*height_part:row*height_part+height_part, col*width_part:col*width_part+width_part]
            img[row*height_part:row*height_part+height_part, col*width_part:col*width_part+width_part] = global_thresholding(img[row*height_part:row*height_part+height_part, col*width_part:col*width_part+width_part])
        
    return img   

def yen_thresholding(img):   
#     img = rgb2gray(img)
#     img = img_as_ubyte(img)

    global_thresh = threshold_yen(img)
    binary_global = img > global_thresh

    return binary_global  

def multi_thresholding(img):   


    global_thresh = threshold_yen(img)
    binary_global = img > global_thresh

    return binary_global  



In [10]:
# mu, sigma = 100, 15
# img = io.imread('..\data\ocr1\img_1.jpg')
# img = rgb2gray(img)
# print(img)
# reference = np.zeros(img.shape)
# height = int(reference.shape[0]/3)
# reference[:height,:] = 0.0
# reference[height:height*2,:] = 1.0
# reference[height*2:,:] = 0.5
# matched = match_histograms(img, reference, multichannel=False)

# # plt.hist(img.ravel(), 256, density=True)
# # plt.show()
# io.imshow(matched)

In [7]:
def to_binary_sheet_and_background(img):
    img = gaussian(img)
    # Contrast stretching
    p2, p98 = np.percentile(img, (2, 98))
    img = rescale_intensity(img, in_range=(p2, p98))


    # dzielimy obraz na 4 części i w nich loklanie wykonujemy thresholding metodą otsu
    width, height = img.shape
    img[:int(height/2), :int(width/2)] = global_thresholding(img[:int(height/2), :int(width/2)])
    img[:int(height/2), int(width/2):] = global_thresholding(img[:int(height/2), int(width/2):])
    img[int(height/2):, :int(width/2)] = global_thresholding(img[int(height/2):, :int(width/2)])
    img[int(height/2):, int(width/2):] = global_thresholding(img[int(height/2):, int(width/2):])
    
    return img

def add_border_to_image(image, width_of_border=30, intensity=0.0):
    (h, w) = image.shape
    new_image_left_border = np.full((h,w+width_of_border), fill_value=intensity)
    new_image_left_border[:,width_of_border:] = image
    image = new_image_left_border
    
    (h, w) = image.shape
    new_image_right_border = np.full((h,w+width_of_border), fill_value=intensity)
    new_image_right_border[:,:-width_of_border] = image
    image = new_image_right_border
    
    (h, w) = image.shape
    new_image_bot_border = np.full((h+width_of_border,w), fill_value=intensity)
    new_image_bot_border[:-width_of_border,:] = image
    image = new_image_bot_border
    
    (h, w) = image.shape
    new_image_top_border = np.full((h+width_of_border,w), fill_value=intensity)
    new_image_top_border[width_of_border:,:] = image
    image = new_image_top_border
    return image


In [57]:
# wykrywanie kartki przez thresholding
source_path = Path('../data/ocr1/')
save_path = Path('../data/partial_results')
save_path.mkdir(parents=True, exist_ok=True)

images_paths = source_path.glob("*.jpg")
for image_path in images_paths:
#     image_path = "..\data\ocr1\img_16.jpg"
    print(image_path)
    img = io.imread(image_path)
    img = rgb2gray(img)

    img = to_binary_sheet_and_background(img)
#     io.imsave(arr=img_as_ubyte(img), fname=save_path / (image_path.stem+'.png'))
    io.imshow(img)
    break

..\data\ocr1\img_16.jpg


In [22]:
# wykrywanie kartki przez thresholding global_v2
source_path = Path('../data/ocr1/')
save_path = Path('../data/partial_results/global_v2')
save_path.mkdir(parents=True, exist_ok=True)

images_paths = source_path.glob("*.jpg")
for image_path in images_paths:
#     image_path = "..\data\ocr1\img_15.jpg"
    print(image_path)
    img = io.imread(image_path)
    img = rgb2gray(img)

    img = gaussian(img)
    # Contrast stretching
    p2, p98 = np.percentile(img, (2, 98))
    img = rescale_intensity(img, in_range=(p2, p98))
    img = global_thresholding_v2(img, rows=5, columns=2)
    
    io.imsave(arr=img_as_ubyte(img), fname=save_path / (image_path.stem+'.png'))
#     io.imshow(img)
#     break

..\data\ocr1\img_1.jpg
..\data\ocr1\img_10.jpg
..\data\ocr1\img_11.jpg
..\data\ocr1\img_12.jpg
..\data\ocr1\img_13.jpg
..\data\ocr1\img_14.jpg
..\data\ocr1\img_15.jpg
..\data\ocr1\img_16.jpg
..\data\ocr1\img_17.jpg
..\data\ocr1\img_18.jpg
..\data\ocr1\img_19.jpg
..\data\ocr1\img_2.jpg
..\data\ocr1\img_20.jpg
..\data\ocr1\img_21.jpg
..\data\ocr1\img_22.jpg
..\data\ocr1\img_23.jpg
..\data\ocr1\img_24.jpg
..\data\ocr1\img_25.jpg
..\data\ocr1\img_26.jpg
..\data\ocr1\img_27.jpg
..\data\ocr1\img_28.jpg
..\data\ocr1\img_29.jpg
..\data\ocr1\img_3.jpg
..\data\ocr1\img_4.jpg
..\data\ocr1\img_5.jpg
..\data\ocr1\img_6.jpg
..\data\ocr1\img_7.jpg
..\data\ocr1\img_8.jpg
..\data\ocr1\img_9.jpg


In [149]:
source_path = Path('../data/ocr1/')
save_path = Path('../data/partial_results/canny')
save_path.mkdir(parents=True, exist_ok=True)

images_paths = source_path.glob("*.jpg")
for image_path in images_paths:
#     image_path = "..\data\ocr1\img_15.jpg"
    print(image_path)
    img = io.imread(image_path)
    img = rgb2gray(img)

#     img = gaussian(img)
    # Contrast stretching
#     p2, p98 = np.percentile(img, (2, 98))
#     img = rescale_intensity(img, in_range=(p2, p98))
    img = img_as_ubyte(img)
    img = canny(img, low_threshold=100, high_threshold=200)
    
#     io.imsave(arr=img_as_ubyte(img), fname=save_path / (image_path.stem+'.png'))
    io.imshow(img)
    break

..\data\ocr1\img_1.jpg


In [80]:
#wykrywanie rogów kartki przez przybliżanie konturu

source_path = Path('../data/partial_results/global_v2')
save_path = Path('../data/partial_results/approx_poly')
save_path.mkdir(parents=True, exist_ok=True)

images_paths = source_path.glob("*.png")
for image_path in images_paths:
#     image_path = "..\data\ocr1\img_15.jpg"
    print(image_path)
    image = io.imread(image_path)
    image = closing(image, square(7))
    
    image = add_border_to_image(image, width_of_border=100)
    cnts = find_contours(image, level=10)

    biggest_contour = sorted(cnts, key=len, reverse=True)[0]

    image2 = approximate_polygon(biggest_contour, tolerance=len(biggest_contour)/20)
    image = gray2rgb(image)
    for row, col in image2:
    #     image[int(row), int(col)] = (255,0,0)
        y, x = draw.circle(int(row), int(col), radius=10)
        image[y,x] = (255,0,0)
    
    
    
    io.imsave(arr=(image), fname=save_path / (image_path.stem+'.png'))


..\data\partial_results\global_v2\img_1.png


Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.


..\data\partial_results\global_v2\img_10.png


Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.


..\data\partial_results\global_v2\img_11.png


...

Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.


..\data\partial_results\global_v2\img_8.png


Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.


..\data\partial_results\global_v2\img_9.png


Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.


In [182]:
# wykrywanie kartki przez histogram
source_path = Path('../data/partial_results/canny_adam/')
save_path = Path('../data/partial_results/histogram_canny_adam')
save_path.mkdir(parents=True, exist_ok=True)

images_paths = source_path.glob("*.png")
for image_path in images_paths:
#     image_path = "..\data\ocr1\img_14.jpg"
    print(image_path)
    img = io.imread(image_path)
    img = rgb2gray(img)
#     img = util.invert(img)
#     io.imshow(img)
    
    sum_of_rows = np.sum(img, axis=1) 
    print(np.max(img))
    mean_row_value = np.mean(sum_of_rows)
    for i, row in enumerate(img):
        if np.sum(row) < mean_row_value:
            row = np.zeros(row.shape)
            img[i] = row
            
    img = img.T
    sum_of_columns = np.sum(img, axis=1)
    mean_column_value = np.mean(sum_of_columns)
    for i, row in enumerate(img):
        if np.sum(row) < mean_column_value:
            row = np.zeros(row.shape)
            img[i] = row
    
    img = img.T
    
    for i, row in enumerate(img):
        if np.mean(row) > 0.0:
            img[i:i+1, :] = 1 

    ############## usuwanie góra/dół ##############
#     for i, row in enumerate(img):
#         if np.mean(row) > 0.0:
#             img = img[i:]
#             break
    
#     img = transform.rotate(img, 180)
#     for i, row in enumerate(img):
#         if np.mean(row) > 0.0:
#             img = img[i:]
#             break
    
#     # usuwanie prawo/lewo
#     img = img.T
    
#     for i, row in enumerate(img):
#         if np.mean(row) > 0.0:
#             img = img[i:]
#             break
    
#     img = transform.rotate(img, 180)
#     for i, row in enumerate(img):
#         if np.mean(row) > 0.0:
#             img = img[i:]
#             break
            
#     img = img.T
    ##########################################
    
    
    print(img.shape)
#     plt.hist(sum_of_rows, bins=len(sum_of_rows))
#     plt.show()

#     io.imsave(arr=img_as_ubyte(img), fname=save_path / (image_path.stem+'.png'))
    io.imshow(img)
    break

..\data\partial_results\canny_adam\file1.png
255
(2560, 1536)


In [90]:
print(np.mean(sum_of_rows))
pos = np.arange(len(sum_of_rows))
plt.barh(pos, sum_of_rows,
                     align='center',
                     height=0.5)

plt.show()

811.3445321124386


In [84]:
arr = np.array([[1,2,3],
               [4,5,6],
               [7,8,9]])
np.sum(arr,axis=1)

array([ 6, 15, 24])

In [49]:
# wycinanie największego konturu z opencv
import numpy as np
import cv2
DEBUG = False

def show_window(name, image, debug=False):
    cv2.imwrite("./assets/output/" + name + ".png", image)
    if debug or DEBUG:
        cv2.namedWindow(name, cv2.WINDOW_NORMAL)
        cv2.resizeWindow(name, (600, 600))
        cv2.imshow(name, image)
        cv2.waitKey(0)
        
def get_biggest_intensity_contour(contours):
    sorted_contours = sorted(contours, key=cv2.contourArea, reverse=False)
    biggest_contour = sorted_contours[0]
    return biggest_contour

def get_rotated_image_from_contour(img, contour):
    """ Returns a rotated version of img based on cv2.minAreaRect of contour.
        First side, (i.e most left to top edge) is always "Width" from minAreaRect.
        If our width > height, we know we have the sheet rotated to the right. 
        We need to shift our angle and width 90 degree in order to get the correct
        coordinates from cv2.boxPoints
    """
    rotated_rect = cv2.minAreaRect(contour)

    # Get the center x,y and width and height.
    x_center = int(rotated_rect[0][0])
    y_center = int(rotated_rect[0][1])
    width = int(rotated_rect[1][0])
    height = int(rotated_rect[1][1])
    angle_degrees = rotated_rect[2]

    if(width > height):
        temp_height = height
        height = width
        width = temp_height
        angle_degrees = 90 + angle_degrees

    # Reassign rotated rect with updated values
    rotated_rect = ((x_center, y_center), (width, height), angle_degrees)
    # Find the 4 (x,y) coordinates for the rotated rectangle, order: bl, tl,tr, br
    rect_box_points = cv2.boxPoints(rotated_rect)

    img_debug_contour = img.copy()
    cv2.drawContours(img_debug_contour, [contour], 0, (0, 0, 255), 3)
    show_window('biggest_contour', img_debug_contour)

    img_debug = img.copy()
    cv2.drawContours(img_debug, [np.int0(rect_box_points)], 0, (0, 0, 255), 3)
    show_window('min_area_rect_original_image', img_debug)

    # Prepare for rotation transformation
    src_pts = rect_box_points.astype("float32")
    dst_pts = np.array([
        [0, height-1],  # Bottom Left
        [0, 0],  # Top Left
        [width-1, 0],  # Top Right
    ], dtype="float32")

    # Affine rotation transformation
    ROTATION_MAT = cv2.getAffineTransform(src_pts[:3], dst_pts)
    return cv2.warpAffine(
        img, ROTATION_MAT, (width, height))


source_path = Path('../data/ocr1/')
save_path = Path('../data/partial_results/CV2_2')
save_path.mkdir(parents=True, exist_ok=True)

images_paths = source_path.glob("*.jpg")
for image_path in images_paths:
#     image_path = "..\data\ocr1\img_16.jpg"
    print(image_path)
    img = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)

    kernel = np.ones((10,10),np.uint8)
    img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
    ret,thresh = cv2.threshold(img,127,255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    print(len(contours))
    biggest_contour = get_biggest_intensity_contour(contours)
    img_raw_yatzy_sheet = get_rotated_image_from_contour(img, biggest_contour)
    
    cv2.imwrite(str(save_path / (image_path.stem+'.png')), img_raw_yatzy_sheet)
#     break

# cv2.namedWindow("elo",cv2.WINDOW_NORMAL)
# cv2.resizeWindow("elo", int(img_raw_yatzy_sheet.shape[1]/2), int(img_raw_yatzy_sheet.shape[0]/2))
# cv2.imshow("elo", img_raw_yatzy_sheet)

..\data\ocr1\img_1.jpg
1
..\data\ocr1\img_10.jpg
1
..\data\ocr1\img_11.jpg
1
..\data\ocr1\img_12.jpg
1
..\data\ocr1\img_13.jpg
1
..\data\ocr1\img_14.jpg
1
..\data\ocr1\img_15.jpg
1
..\data\ocr1\img_16.jpg
1
..\data\ocr1\img_17.jpg
1
..\data\ocr1\img_18.jpg
1
..\data\ocr1\img_19.jpg
1
..\data\ocr1\img_2.jpg
1
..\data\ocr1\img_20.jpg
1
..\data\ocr1\img_21.jpg
1
..\data\ocr1\img_22.jpg
1
..\data\ocr1\img_23.jpg
1
..\data\ocr1\img_24.jpg
1
..\data\ocr1\img_25.jpg
1
..\data\ocr1\img_26.jpg
1
..\data\ocr1\img_27.jpg
1
..\data\ocr1\img_28.jpg
1
..\data\ocr1\img_29.jpg
1
..\data\ocr1\img_3.jpg
1
..\data\ocr1\img_4.jpg
1
..\data\ocr1\img_5.jpg
1
..\data\ocr1\img_6.jpg
1
..\data\ocr1\img_7.jpg
1
..\data\ocr1\img_8.jpg
1
..\data\ocr1\img_9.jpg
1


In [33]:

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from skimage import data
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
from skimage.color import label2rgb


image = io.imread('..\data\partial_results\img_3.png')
image = closing(image, square(10))
# apply threshold
# thresh = threshold_otsu(image)
# bw = closing(image > thresh, square(3))

# remove artifacts connected to image border
# cleared = clear_border(bw)

# label image regions
label_image = label(image)
# to make the background transparent, pass the value of `bg_label`,
# and leave `bg_color` as `None` and `kind` as `overlay`
image_label_overlay = label2rgb(label_image, image=image, bg_label=0)

fig, ax = plt.subplots(figsize=(10, 6))
ax.imshow(image_label_overlay)

for region in regionprops(label_image):
    # take regions with large enough areas
    if region.area >= 100000:
        # draw rectangle around segmented coins
        minr, minc, maxr, maxc = region.bbox
        rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr,
                                  fill=False, edgecolor='red', linewidth=2)
        ax.add_patch(rect)

ax.set_axis_off()
plt.tight_layout()
plt.show()

In [56]:
# kwadrat = np.array([[0,0,0,0,0],
#            [0,1,1,1,0],
#            [0,1,0,1,0],
#            [0,1,1,1,0],
#            [0,0,0,0,0]])
# x, y = np.where(kwadrat==1)
# contour_pos = np.array(list(zip(x,y)))
# contour_pos = np.array([np.array(a) for a in contour_pos])
# contour_pos
# contour_pos = np.reshape(contour_pos, (-1, 1, 2))


# import numpy as np
# import cv2 as cv
# img = cv.imread('..\data\partial_results\img_1.png',0)
# kernel = np.ones((10,10),np.uint8)
# img = cv.morphologyEx(img, cv.MORPH_CLOSE, kernel)
# ret,thresh = cv.threshold(img,127,255,0)
# contours,hierarchy = cv.findContours(thresh, 1, 2)
# print((contours[0]))
# cnt = contours[0]
# print(contour_pos)
# M = cv.moments(contour_pos)
# print( M['m00'] )

In [105]:
def get_points_and_angles_in_image(image):
    possible_baseline_points = find_corners(image)
    angles_of_points = list()
    for point in possible_baseline_points:
        angle = get_angle_of_point_using_hough(image, point)
        angles_of_points.append(angle)

    return possible_baseline_points, angles_of_points

def find_corners(image):
    contours = find_contours(image, level=10)
    most_n_of_points = len(contours[0])
    most_n_of_points_countour_index = 0
    for i, contour in enumerate(contours):
        temp = len(contour)
        if temp > most_n_of_points:
            most_n_of_points = temp
            most_n_of_points_countour_index = i
    
    contour = contours[most_n_of_points_countour_index]
    
    appr_pol = list(approximate_polygon(contour, tolerance=10)[:-1]) 
    appr_pol = np.around(np.array(appr_pol))
    appr_pol = corner_subpix(image, appr_pol, window_size=50)
    appr_pol = appr_pol[~np.isnan(appr_pol).any(axis=1)]
    appr_pol = list(appr_pol)
#     appr_pol = appr_pol+appr_pol[:1]
    return appr_pol

def get_angle_of_point_using_hough(image, point):
    slice_of_image, _ = slice_image_in_radius(image, point, radius=100)
    slice_of_image = opening(slice_of_image, square(1))
    list_of_angles = get_list_of_different_angles_in_image_using_hough(image=slice_of_image)
#     points = corner_peaks(corner_harris(slice_of_image), min_distance=10)
#     if len(points) > 0:
#         point = points[0]
    if len(list_of_angles) == 2:
        angle_between_lines = abs(list_of_angles[0]-list_of_angles[1])
        return angle_between_lines
    
    return None

def slice_image_in_radius(image, point, radius):
    (height, width) = point
    height = int(height)
    width = int(width)
    (image_height, image_width) = image.shape
    

    point_height_in_sliced_image = radius+1
    point_width_in_sliced_image = radius+1
    if height-radius < 0 :
        point_height_in_sliced_image = radius - (radius-height)
    if width-radius < 0:
        point_width_in_sliced_image = radius - (radius-width)
    middle_point = (point_height_in_sliced_image, point_width_in_sliced_image)

    right_width_min = clamp(width-radius-1, 0, image_width)
    right_width_max = clamp(width+radius+1, 0, image_width)
    right_height_min = clamp(height-radius-1, 0, image_height)
    right_height_max = clamp(height+radius+1, 0, image_height)
    slice_of_image = image[right_height_min:right_height_max, right_width_min:right_width_max]
    return slice_of_image, middle_point

def get_list_of_different_angles_in_image_using_hough(image):
    edges = canny(image)
    lines = probabilistic_hough_line(edges, 
                                     threshold=10, 
                                     line_length=50,
                                     line_gap=1)
    list_of_angles = [get_angle_of_line(line) for line in lines]
    list_of_angles_no_duplicates = list()
#     print(len(lines))
    
    for angle in list_of_angles:
        add = True
        for added_angle in list_of_angles_no_duplicates:
            if compare_2_numbers_with_range(added_angle, angle, 5):
                add = False
        if add:
            list_of_angles_no_duplicates.append(angle)
                
    # np. kąty -176 i 180 są obok siebie, zależą od zwrotu, to samo z kątem 90
    # tutaj usuwamy takie przypadki, żeby mieć na penwo różne kąty
    exists_90 = False
    exists_180 = False
    modified_list_of_angles = list()
    for angle in list_of_angles_no_duplicates:
        if compare_2_numbers_with_range(90, abs(angle), 5):
            if exists_90 == False:
                exists_90 = True
                modified_list_of_angles.append(angle)
        elif compare_2_numbers_with_range(180, abs(angle), 5) or compare_2_numbers_with_range(0, abs(angle), 5):
            if exists_180 == False:
                exists_180 = True
                modified_list_of_angles.append(angle)
        else:
            modified_list_of_angles.append(angle)
    
    return modified_list_of_angles

def clamp(n, smallest, largest): 
    return max(smallest, min(n, largest))

def get_angle_of_line(line):
    (point1, point2) = line
    angle = math.atan2(point2[1] - point1[1], point2[0] - point1[0]) * 180 / np.pi
    return angle #if angle >= 0 else 180+angle

def compare_2_numbers_with_range(n1, n2, range=3):
    difference = abs(n1-n2)
    return difference <= range

In [115]:
image = io.imread('..\data\partial_results\img_4.png')
image = closing(image, square(10))
image = add_border_to_image(image, width_of_border=100)
slice_of_image, _ = slice_image_in_radius(rgb2gray(image), (382, 150), radius=100)
io.imshow(image)

In [25]:
# wyszukiwanie rogów kartki
from matplotlib import pyplot as plt

from skimage import data
from skimage.feature import corner_harris, corner_subpix, corner_peaks
from skimage.transform import warp, AffineTransform
from skimage.draw import ellipse
from skimage.measure import label, regionprops
source_path = Path('../data/partial_results/')
save_path = Path('../data/partial_results/corners2')
save_path.mkdir(parents=True, exist_ok=True)

images_paths = source_path.glob("*.png")
for image_path in images_paths:
    image_path = Path('..\data\partial_results\img_2.png')
    print(image_path) 
    image = io.imread(image_path)
#     image = opening(image, square(10))
    image = closing(image, square(10))
#     io.imshow(image)
#     break
    
    def get_area(region):
        return region.area
    
    image = add_border_to_image(image, width_of_border=100)
    label_image = label(image)
    regions = regionprops(label_image)
    biggest_region = sorted(regions, key=get_area, reverse=True)[0]

    coords = corner_peaks(corner_harris(image, k=0.0), min_distance=11, threshold_rel=0.02)
    coords_subpix = corner_subpix(image, coords, window_size=13)
#     points, angles = get_points_and_angles_in_image(image)

#     print(np.array(points))
#     print(angles)

    coords_subpix = [el for el in coords_subpix if el in biggest_region.coords]
    image = gray2rgb(image)
    app = approximate_polygon(np.array(coords_subpix), tolerance=13)
    print(app)
    for point in app:
        y, x = draw.circle(point[0], point[1], radius=10)
        image[y,x] = (255,0,0)
            
            
#     for point, angle in zip(points, angles):
#         if (angle is not None) and (compare_2_numbers_with_range(90, angle, 10) or compare_2_numbers_with_range(270, angle, 10)):
#             y, x = draw.circle(point[0], point[1], radius=10)
#             image[y,x] = (255,0,0)
#     io.imsave(arr=image, fname=save_path / (image_path.stem+'.png'))

    io.imshow(image)
    break





..\data\partial_results\img_2.png
[[ 214  137]
 [ 214  138]
 [ 214  139]
 ...
 [2582  196]
 [2582  197]
 [2582  198]]
[[ 172.         1625.        ]
 [ 221.38383838 1021.        ]
 [ 227.38383838 1578.        ]
 [ 695.          147.10204082]
 [ 945.          116.        ]
 [ 969.          140.        ]
 [2033.          167.38383838]
 [2537.61616162 1174.        ]
 [2561.61616162  730.        ]]


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [5]:
io.imshow(io.imread(Path('..\data\partial_results\img_2.png')))

In [60]:
import numpy as np

from skimage.transform import hough_line, hough_line_peaks
from skimage.feature import canny
from skimage import data

import matplotlib.pyplot as plt
from matplotlib import cm

image_path = "..\data\ocr1\img_15.jpg"
print(image_path)
image = io.imread(image_path)
image = rgb2gray(image)
image = canny(image)
image = closing(image, square(5))

tested_angles = np.linspace(-np.pi / 2, np.pi / 2, 360)
h, theta, d = hough_line(image, theta=tested_angles)

lines = probabilistic_hough_line(image, threshold=10, line_length=5,
                                 line_gap=3)

# Generating figure 1
fig, axes = plt.subplots(1, 3, figsize=(15, 6))
ax = axes.ravel()

ax[0].imshow(image, cmap=cm.gray)
ax[0].set_title('Input image')
ax[0].set_axis_off()

ax[1].imshow(np.log(1 + h),
             extent=[np.rad2deg(theta[-1]), np.rad2deg(theta[0]), d[-1], d[0]],
             cmap=cm.gray, aspect=1/1.5)
ax[1].set_title('Hough transform')
ax[1].set_xlabel('Angles (degrees)')
ax[1].set_ylabel('Distance (pixels)')
ax[1].axis('image')

ax[2].imshow(image, cmap=cm.gray)
origin = np.array((0, image.shape[1]))
for _, angle, dist in zip(*hough_line_peaks(h, theta, d)):
    y0, y1 = (dist - origin * np.cos(angle)) / np.sin(angle)
    ax[2].plot(origin, (y0, y1), '-r')
ax[2].set_xlim(origin)
ax[2].set_ylim((image.shape[0], 0))
ax[2].set_axis_off()
ax[2].set_title('Detected lines')

plt.tight_layout()
plt.show()

..\data\ocr1\img_15.jpg


In [97]:
import numpy as np

from skimage.transform import hough_line, hough_line_peaks
from skimage.feature import canny
from skimage import data
from skimage.transform import probabilistic_hough_line

import matplotlib.pyplot as plt
from matplotlib import cm

image_path = "..\data\ocr1\img_1.jpg"
print(image_path)
image = io.imread(image_path)
image = rgb2gray(image)

# Line finding using the Probabilistic Hough Transform
image = filters.scharr(image)
# image = closing(image, square(5))
# edges = canny(image) #, 2, 1, 25


io.imshow(image)

# # edges = dilation(edges, square(5))
# lines = probabilistic_hough_line(edges, threshold=10, line_length=500,
#                                  line_gap=10)

# # Generating figure 2
# fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharex=True, sharey=True)
# ax = axes.ravel()

# ax[0].imshow(image, cmap=cm.gray)
# ax[0].set_title('Input image')

# ax[1].imshow(edges, cmap=cm.gray)
# ax[1].set_title('Canny edges')

# ax[2].imshow(edges * 0)
# for line in lines:
#     p0, p1 = line
#     ax[2].plot((p0[0], p1[0]), (p0[1], p1[1]))
# ax[2].set_xlim((0, image.shape[1]))
# ax[2].set_ylim((image.shape[0], 0))
# ax[2].set_title('Probabilistic Hough')

# for a in ax:
#     a.set_axis_off()

# plt.tight_layout()
# plt.show()

..\data\ocr1\img_1.jpg


In [59]:
import numpy as np

from skimage.transform import hough_line, hough_line_peaks
from skimage.feature import canny
from skimage import data
from skimage.transform import probabilistic_hough_line

import matplotlib.pyplot as plt
from matplotlib import cm


image_path = Path('..\data\ocr1\img_8.jpg')
image = io.imread(image_path)
image = rgb2gray(image)
# image = local_thresholding(image)
# image = opening(image, square(5))
image = gaussian(image)
edges = canny(image)

lines = probabilistic_hough_line(edges, threshold=10, line_length=100,
                                 line_gap=10)


# Generating figure 2
fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(image, cmap=cm.gray)
ax[0].set_title('Input image')

ax[1].imshow(edges, cmap=cm.gray)
ax[1].set_title('Canny edges')

ax[2].imshow(edges * 0)
for line in lines:
    p0, p1 = line
    ax[2].plot((p0[0], p1[0]), (p0[1], p1[1]))
ax[2].set_xlim((0, image.shape[1]))
ax[2].set_ylim((image.shape[0], 0))
ax[2].set_title('Probabilistic Hough')

for a in ax:
    a.set_axis_off()

plt.tight_layout()
plt.show()

In [87]:
import numpy as np

from skimage.transform import hough_line, hough_line_peaks
from skimage.feature import canny
from skimage import data

import matplotlib.pyplot as plt
from matplotlib import cm


image_path = Path('..\data\ocr1\img_8.jpg')
image = io.imread(image_path)
image = rgb2gray(image)
image = add_border_to_image(image,width_of_border=100)
image = img_as_bool(image)

# image = local_thresholding(image)
# image = opening(image, square(5))

image = canny(image)
# image = gaussian(image)
io.imshow(image)

# Classic straight-line Hough transform
# Set a precision of 0.5 degree.
tested_angles = np.linspace(-np.pi / 2, np.pi / 2, 360)
h, theta, d = hough_line(image, theta=tested_angles)

# Generating figure 1
fig, axes = plt.subplots(1, 3, figsize=(15, 6))
ax = axes.ravel()

ax[0].imshow(image, cmap=cm.gray)
ax[0].set_title('Input image')
ax[0].set_axis_off()

ax[1].imshow(np.log(1 + h),
             extent=[np.rad2deg(theta[-1]), np.rad2deg(theta[0]), d[-1], d[0]],
             cmap=cm.gray, aspect=1/1.5)
ax[1].set_title('Hough transform')
ax[1].set_xlabel('Angles (degrees)')
ax[1].set_ylabel('Distance (pixels)')
ax[1].axis('image')

ax[2].imshow(image, cmap=cm.gray)
origin = np.array((0, image.shape[1]))
for _, angle, dist in zip(*hough_line_peaks(h, theta, d)):
    y0, y1 = (dist - origin * np.cos(angle)) / np.sin(angle)
    ax[2].plot(origin, (y0, y1), '-r')
ax[2].set_xlim(origin)
ax[2].set_ylim((image.shape[0], 0))
ax[2].set_axis_off()
ax[2].set_title('Detected lines')

plt.tight_layout()
plt.show()

In [8]:
# opecv - shape detector
# import the necessary packages
import cv2
import imutils
class ShapeDetector:
    def __init__(self):
        pass
    def detect(self, c):
        # initialize the shape name and approximate the contour
        shape = "unidentified"
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.04 * peri, True)
        # if the shape is a triangle, it will have 3 vertices
        if len(approx) == 3:
            shape = "triangle"
        # if the shape has 4 vertices, it is either a square or
        # a rectangle
        elif len(approx) == 4:
            # compute the bounding box of the contour and use the
            # bounding box to compute the aspect ratio
            (x, y, w, h) = cv2.boundingRect(approx)
            ar = w / float(h)
            # a square will have an aspect ratio that is approximately
            # equal to one, otherwise, the shape is a rectangle
            shape = "square" if ar >= 0.95 and ar <= 1.05 else "rectangle"
        # if the shape is a pentagon, it will have 5 vertices
        elif len(approx) == 5:
            shape = "pentagon"
        # otherwise, we assume the shape is a circle
        else:
            shape = "circle"
        # return the name of the shape
        return shape
    
# find contours in the thresholded image and initialize the
# shape detector
thresh = cv2.imread('..\data\partial_results\img_8.png', cv2.THRESH_BINARY)
resized = imutils.resize(image, width=300)
ratio = thresh.shape[0] / float(resized.shape[0])
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
sd = ShapeDetector()

for c in cnts:
    # compute the center of the contour, then detect the name of the
    # shape using only the contour
    M = cv2.moments(c)
    if M["m00"] == 0:
        continue
    cX = int((M["m10"] / M["m00"]) * ratio)
    cY = int((M["m01"] / M["m00"]) * ratio)
    shape = sd.detect(c)
    # multiply the contour (x, y)-coordinates by the resize ratio,
    # then draw the contours and the name of the shape on the image
    c = c.astype("float")
    c *= ratio
    c = c.astype("int")
    cv2.drawContours(image, [c], -1, (0, 255, 0), 2)
    cv2.putText(image, shape, (cX, cY), cv2.FONT_HERSHEY_SIMPLEX,
        0.5, (255, 255, 255), 2)
    # show the output image
    cv2.imshow("Image", image)
    cv2.waitKey(0)

NameError: name 'ratio' is not defined